In [55]:
import pandas as pd
import numpy as np
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [56]:
df = pd.read_pickle('/kaggle/input/preprocessed-dataset-pkl/preprocessed_dataset.pkl')

In [57]:
df.head(3)

,imdbID,title,fullplot,meta_tags,all tags
2008,35423,Kate & Leopold,kate actor brother live ny 21st centuri exboyf...,comedy fantasy romance jamesmangold megryan hu...,kate actor brother live ny 21st centuri exboyf...
7011,64994,Larks on a String,pragu earli 1950 bourgeoi element reeduc work ...,comedy drama romance jir�menzel rudolfhrus�nsk...,pragu earli 1950 bourgeoi element reeduc work ...
10044,81145,Me and the Kid,two excon,comedy crime drama dancurtis dannyaiello alexz...,two excon comedy crime drama dancurtis dannyai...


In [58]:
df.reset_index(drop=True,inplace=True)

In [59]:

tfidf_vectorizer1 = TfidfVectorizer( max_features=20000,ngram_range=(1,4),max_df=0.4)
feature_array = tfidf_vectorizer1.fit_transform(df["all tags"]).toarray()

In [60]:
feature_array.shape

(25347, 20000)

In [61]:
import cupy as cp
from cupyx.scipy.sparse import csr_matrix
from cupyx.scipy.sparse import spmatrix

def compute_cosine_similarity_in_batches(feature_array, batch_size=2000):

    feature_array_gpu = cp.array(feature_array, dtype=cp.float32)
    
    num_samples = feature_array_gpu.shape[0]
    similarity_matrix_gpu = cp.zeros((num_samples, num_samples), dtype=cp.float32)
    
    
    for start_idx in range(0, num_samples, batch_size):
        
        end_idx = min(start_idx + batch_size, num_samples)
        batch_features_gpu = feature_array_gpu[start_idx:end_idx]
        
       
        batch_norms = cp.linalg.norm(batch_features_gpu, axis=1, keepdims=True)
        feature_norms = cp.linalg.norm(feature_array_gpu, axis=1, keepdims=True)
        
        batch_similarity_gpu = cp.dot(batch_features_gpu / batch_norms, feature_array_gpu.T / feature_norms.T)
        
        
        similarity_matrix_gpu[start_idx:end_idx] = batch_similarity_gpu.astype(cp.float32)
    
    
    similarity_matrix = cp.asnumpy(similarity_matrix_gpu)
    
    return similarity_matrix

similarity_matrix = compute_cosine_similarity_in_batches(feature_array, batch_size=2000)

In [62]:
def recommend_movie(movie):
    movie_index = df[df['title']==movie].index[0]
    distance = similarity_matrix[movie_index]
    movie_list = sorted(enumerate(distance),key=lambda x: x[1],reverse=True)[1:11]
    for i in movie_list:
        print(df.iloc[i[0],1])

In [67]:
recommend_movie('The Lord of the Rings: The Return of the King')

The Lord of the Rings: The Two Towers
Zenon: The Zequel
RKO Production 601: The Making of 'Kong, the Eighth Wonder of the World'
The Hobbit: The Desolation of Smaug
Without a Paddle
The Hobbit: An Unexpected Journey
The Lord of the Rings: The Fellowship of the Ring
Underworld: Rise of the Lycans
Under the Mountain
The Secret of Moonacre
